# INBOX 
See all incoming messages here

In [0]:
!pip uninstall s3i

In [0]:
!pip install https://git.rwth-aachen.de/kwh40/s3i/-/jobs/artifacts/master/raw/public/s3i-0.4-py3-none-any.whl?job=wheel

In [0]:
from s3i import IdentityProvider, Broker, UserMessage, TokenType, Key, Directory
import time 
import base64 

In [0]:
USERNAME = "KWH-Team" #"adelbert" #input('[S3I]: Please enter the username:')
PASSWORD = "V4yuJ31izh0GCH36954O" #"adelbert" #"V4yuJ31izh0GCH36954O" #getpass.getpass('[S3I]: Please enter the password:')

In [0]:
CLIENT = "s3i:9b34f12b-f0ab-4f13-b0cc-82202e7a9770" #"s3i:4f076127-d9e2-477d-8d42-4453225bfaaa" #"s3i:3e7cfc9b-2fda-4c56-a4f0-b5e8f34b6da6" #"s3i:e09825e5-a872-4541-9ef4-31ceab014371" #input('[S3I]: Please enter the thing id of this HMI:') #
SECRET = "a1f025f3-687c-44ae-8b79-cd7ebdd076c8" #"5ad8387a-5183-471b-8dfe-da2c7c62557f" #"346e9867-e278-4ba3-b336-1421e5abd594" # getpass.getpass('[S3I]: Please enter the secret:') # 

In [0]:
IDP = IdentityProvider(grant_type='password', identity_provider_url="https://idp.s3i.vswf.dev/", realm='KWH',
                                             client_id=CLIENT, client_secret=SECRET, username=USERNAME, password=PASSWORD)

In [0]:
def callback(ch, method, properties, body):
    print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S",
                                   time.localtime()) + "]: A new message has been received")
    body_str = body.decode('utf8').replace("'", '"')  # convert bytes to str
    uMsg = UserMessage(msg_blob=body_str)
    
    access_token = IDP.get_token(TokenType.ACCESS_TOKEN)
    dir = Directory(s3i_dir_url="https://dir.s3i.vswf.dev/api/2/", token=access_token)
    """Decrypt the message and verify the signature
    """
    if uMsg.pgpMsg.is_encrypted:
        sec_key = Key(path_demo_dir="/content/", filename="workshop_sec.asc")
        uMsg.decryptAndVerify(sec_key, "", dir)
        #uMsg.decrypt(sec_key.key)
        uMsg.convertPgpToMsg()

    print("[S3I]: Subject of the message: " + uMsg.msg["subject"])
    print("[S3I]: Text of the message:  " + uMsg.msg["text"])
    print("[S3I]: Sender of the message:  :", dir.queryThingIDBased(uMsg.msg["sender"]+"/attributes/name"))

    attachments_list = uMsg.msg["attachments"]
    """
    store the attachment file in specified path
    """
    for attachment in attachments_list:
        with open("/content/received_data/"+attachment["filename"], 'wb') as file:
            decode = base64.b64decode(attachment["data"])
            file.write(decode)
            print("[S3I]: Attachment " + attachment["filename"]
               + " of the message is stored in received_data")
        


In [0]:
access_token = IDP.get_token(TokenType.ACCESS_TOKEN)
broker = Broker(auth_form='Username/Password', username=" ", password=access_token, host="rabbitmq.s3i.vswf.dev")
broker.receive("s3ibs://"+"s3i:bf5a3b54-1ecc-4195-bc55-cb4757b895d0", callback)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
access_token = IDP.get_token(TokenType.ACCESS_TOKEN)
dir = Directory(s3i_dir_url="https://dir.s3i.vswf.dev/api/2/", token=access_token)
a = dir.getPublicKey("s3i:9b34f12b-f0ab-4f13-b0cc-82202e7a9770")
print(a)

In [96]:
import os
import urllib.request
urllib.request.urlretrieve("https://drive.google.com/file/d/1uSRKSnoY76yaJq-c-dnPdKdXwVsaySfy/view?usp=sharing")

('/tmp/tmp7_c_6g7l', <http.client.HTTPMessage at 0x7f89f7619d30>)

In [0]:
!wget -O /content/attachment1.txt https://drive.google.com/file/d/1uSRKSnoY76yaJq-c-dnPdKdXwVsaySfy/view?usp=sharing

In [0]:
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

file_id = created.get('id')

import io
from googleapiclient.http import MediaIoBaseDownload

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, done = downloader.next_chunk()

downloaded.seek(0)
print('Downloaded file contents are: {}'.format(downloaded.read()))

In [0]:
#request = drive_service.files().get_media(fileId=file_id)
import io
from googleapiclient.http import MediaIoBaseDownload
request = "https://drive.google.com/file/d/1uSRKSnoY76yaJq-c-dnPdKdXwVsaySfy/view?usp=sharing"
fh = io.BytesIO()
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print ("Download {}".format(int(status.progress() * 100)))


In [0]:
file_id = created.get('id')

import io
from googleapiclient.http import MediaIoBaseDownload

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, done = downloader.next_chunk()

downloaded.seek(0)
print('Downloaded file contents are: {}'.format(downloaded.read()))